In [ ]:
%matplotlib inline
import os
import sys
sys.path.insert(0, os.getcwd())
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.image as mpimg
import seaborn as sns
import warnings;
warnings.filterwarnings('ignore')

WORKDIR = os.getcwd()
os.chdir(WORKDIR)
print(WORKDIR)
if not os.path.isdir("Figures"):
    os.mkdir("Figures")

In [ ]:
# Standard curves
%run -i biomarkdataparser.py -i rawdata/Assayvalidation_triplicates_rawdata.csv --title "" --number 3 --replicates assay --outdir Standardcurves --outfile "Figures/FigS3_standard_curves_validation" -c -f png

In [ ]:
# Mock community pre-amplification
%run -i biomarkdataparser.py -i rawdata/Assayvalidation_triplicates_rawdata.csv --number 3 --replicates assay --outdir Validation/Mock_community --samplefilter Mock -x Fail NTC -f png --italic y --color --datatoplot copy --transpose --title "" --figsize 6 12 --legendposition -0.1 0.79 --labelfile labelfiles/preamp_labels.csv --outfile "Figures/Fig3_Mock_community"

In [ ]:
# Difference between expected & measured copy numbers in mock communit dilutions
%run -i biomarkdataparser.py -i rawdata/Assayvalidation_triplicates_rawdata.csv --number 3 --replicates assay --outdir Validation/Mock_community_dilution --samplefilter Mock -x Fail preamp -f png --italic y --color --datatoplot "None" --transpose --title "" --figsize 7 12 --legendposition -0.1 0.79 --rawdata

In [ ]:
# Difference between expected & measured copy numbers in mock communit dilutions
dildata = "Validation/Mock_community_dilution/raw_copy_heatmap_data.csv"
df = pd.read_csv(dildata, sep=",", index_col=0)
df = df.drop("4 Mock 10^2")
df = df.replace(np.nan, 0)
# share axes
ncols=6
assays = list(df.columns)
nrows = 4
assays.sort()

fig, ax = plt.subplots(
        nrows=nrows, ncols=ncols, sharex='col', sharey='row',
        figsize=(ncols*2.5, nrows*2))

fig.add_subplot(111, frameon=False)
plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
plt.grid(False)
plt.xlabel("Predicted copies / \u03BCl (" + "$log_{10}$" + ")", fontsize=16, labelpad=10)
plt.ylabel("Measured copies / \u03BCl (" + "$log_{10}$" + ")", fontsize=16, labelpad=10)
ind = 0
for r in range(0, nrows):
    for c in range(0, ncols):         
        q = np.array(df.iloc[:, ind:ind + 3]).flatten().tolist()
        z = q.count(0)
        if z >= 2:
            ey = q[-3::]
            ex = [3, 3, 3]
            y = q[0:-3]
            x = [5, 5, 5, 4, 4, 4]
        else:
            x = [5, 5, 5, 4, 4, 4, 3, 3, 3]
            y = q
            ey = []
            ex = []

        ax[r, c].plot(x, y, '.', c="black", markersize=6)
        ax[r, c].plot(ex, ey, '.', c="red", markersize=6)
        
        ax[r, c].plot([5, 5, 5, 4, 4, 4, 3, 3, 3], [5, 5, 5, 4, 4, 4, 3, 3, 3], c="blue", linewidth=0.8)
        ax[r, c].set_xlim(2, 6)
        ax[r, c].set_ylim(2, 6)

        ax[r, c].set_title(df.columns[ind].split(" ")[0], fontsize=12)

        ax[r, c].xaxis.set_ticks([2, 3, 4, 5, 6]) 
        ax[r, c].yaxis.set_ticks([2, 3, 4, 5, 6])  

        for tick in ax[r, c].xaxis.get_major_ticks():
            tick.label.set_fontsize(10)
        for tick in ax[r, c].yaxis.get_major_ticks():
            tick.label.set_fontsize(10)
        # Triplicates
        ind +=3
    
plt.savefig("Figures/FigS4_predicted_measured_copies.png", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
%run -i biomarkdataparser.py -i rawdata/Assayvalidation_triplicates_rawdata.csv --number 3 --replicates assay --outdir Validation/deltaCq --samplefilter "2 Mock" "5 Mock" --deltact "(preamp)" "Mock" --datatoplot None

In [ ]:
# Delta Cq Table
assay_dict = {}
with open("labelfiles/assay_species.csv") as f:
    reader = csv.reader(f)
    for row in reader:
        assay_dict[row[0]] = row[1]

df = pd.read_csv("Validation/deltaCq/cq_heatmap_data.csv", sep=",", index_col=0)
df = df.T
df.columns = ["Cq before preamplification", "Cq after preamplification"]
df["Delta Cq"] = df["Cq before preamplification"] - df["Cq after preamplification"]
df.index = df.index.map(assay_dict)
df = df.round(decimals=2)
df.index.name = "Species"
df.to_csv("Figures/Tab4_preamp_delta_cq.csv")

In [ ]:
# Standard calibration curves Cheese samples
%run -i biomarkdataparser.py -i rawdata/CheeseSamples_validation_rawdata.csv --number 3 --replicates sample --title "" --outdir Cheese/Standardcurves -c --outfile "Figures/FigS5_cheesesamples_standard_calibration_curves" -f png

In [ ]:
# Inoculated cheese samples
%run -i biomarkdataparser.py -i rawdata/CheeseSamples_validation_rawdata.csv --number 3 --replicates sample --outdir Cheese/adjuncts --samplefilter V18-22-73 -f png --title "" -x PW --italic y --color --transpose --legendposition -0.1 0.788 --labelfile labelfiles/inoculated_cheese_labels.csv

In [ ]:
df2 = pd.read_csv("rawdata/bga_prop_data.csv", sep=",", index_col=0)
df_prop = pd.DataFrame(df2.iloc[3,])
df_prop = df_prop.T
df2.drop("Propionic acid", inplace=True)

spec_pal = ['#1E88E5', '#FFC107', '#004D40']
df2.sort_index(inplace=True, ascending=False)

fig, axs = plt.subplots(
            2, len(df2.columns), figsize=(6, 0.5), sharey='row',
            gridspec_kw={'height_ratios': [4, 1]})

for i, j in enumerate(df2.columns):
    names = df2.index
    values=df2[j]
    axs[0, i].barh(y=names, width=values, height=0.8, color=spec_pal)
    axs[0, i].set_xlim(0, max(df2.loc["Tyramine"]))

    axs[0, i].xaxis.set_ticks_position("top")
    axs[0, i].tick_params(axis="y", length=0, width=0, labelsize=7)
    axs[0, i].tick_params(axis="x", length=2, pad=0, labelsize=4)
    
axs[0, 0].axes.get_yaxis().set_visible(False)
axs[0, 18].yaxis.set_ticks_position("right")

axs[0, 10].set_xlabel("[mg/kg]", labelpad=-35, fontsize=6)
    
for i, j in enumerate(df_prop.columns):
    names = df_prop.index
    values=df_prop[j]
    axs[1, i].barh(y=names, width=values, height=0.8, color="red")
    axs[1, i].set_xlim(0, max(df_prop.loc["Propionic acid"]))
    axs[1, i].xaxis.set_ticks_position("bottom")
    axs[1, i].tick_params(axis="y", length=0, width=0, labelsize=7)
    axs[1, i].tick_params(axis="x", length=2, pad=0, labelsize=4)
    axs[1, i].xaxis.set_ticks([0, 20])
axs[1, 0].axes.get_yaxis().set_visible(False)
axs[1, 18].yaxis.set_ticks_position("right")
    
axs[1, 10].set_xlabel("[mmol/kg]", labelpad=1, fontsize=6)

plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.01, hspace=None)

plt.savefig("Cheese/adjuncts/figures/chem_analysis.png", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 11))
width_ratios=[1.9, 15, 2.8]
height_ratios=[1.5, 12]
spec = gridspec.GridSpec(
                        ncols=3, nrows=2, figure=fig,
                        width_ratios=width_ratios, height_ratios=height_ratios,
                        hspace=0, wspace=0) # 
ax1 = fig.add_subplot(spec[0, 1])
ax2 = fig.add_subplot(spec[1, 0::])

chem = mpimg.imread("Cheese/adjuncts/figures/chem_analysis.png")
heatmap = mpimg.imread('Cheese/adjuncts/figures/qPCR_heatmap_copy.png')

ax1.imshow(chem, origin='upper')
ax2.imshow(heatmap)

ax1.set_frame_on(False)
ax1.patch.set_visible(False)
ax1.set_xticklabels([])
ax1.set_yticklabels([])
ax1.tick_params(axis="both",length=0, width=0., pad=0)

ax2.set_frame_on(False)
ax2.patch.set_visible(False)
ax2.set_xticklabels([])
ax2.set_yticklabels([])
ax2.tick_params(axis="both",length=0, width=0., pad=0)

plt.axis("off")

plt.savefig("Figures/Fig4_cheese_adjuncts.png", dpi=600, bbox_inches='tight')

In [ ]:
# Cheese with defects
%run -i biomarkdataparser.py -i rawdata/CheeseSamples_validation_rawdata.csv --number 3 --replicates sample --outdir Cheese/Praxisproben --typefilter Unknown --samplefilter Commercial --transpose -x "Not relevant" -f png --title "" --labelfile labelfiles/cheese_labels.csv --italic y --color --figsize 4 9 --legendposition -0.1 0.788

In [ ]:
df2 = pd.read_csv("rawdata/bga_prop_praxis.csv", sep=",", index_col=0)
df_prop = pd.DataFrame(df2.iloc[3,])
df_prop = df_prop.T
df2.drop("Propionic acid", inplace=True)
spec_pal = ['#1E88E5', '#FFC107', '#004D40']
df2.sort_index(inplace=True, ascending=False)

fig, axs = plt.subplots(
            2, len(df2.columns), figsize=(1.5, 0.5), sharey='row',
    gridspec_kw={'height_ratios': [4, 1]})

for i, j in enumerate(df2.columns):
    names = df2.index
    values=df2[j]
    axs[0, i].barh(y=names, width=values, height=0.8, color=spec_pal)
    axs[0, i].set_xlim(0, max(df2.loc["Histamine"]))
    axs[0, i].xaxis.set_ticks_position("top")
    axs[0, i].tick_params(axis="y", length=0, width=0, labelsize=6)
    axs[0, i].tick_params(axis="x", length=2, pad=0, labelsize=4)
    axs[0, i].xaxis.set_ticks([0, 500])
axs[0, 0].axes.get_yaxis().set_visible(False)
axs[0, 1].yaxis.set_ticks_position("right")

axs[0, 1].set_xlabel("[mg/kg]", labelpad=-32, fontsize=5)
    
for i, j in enumerate(df_prop.columns):
    names = df_prop.index
    values=df_prop[j]
    axs[1, i].barh(y=names, width=values, height=0.8, color="red")
    axs[1, i].set_xlim(0, max(df_prop.loc["Propionic acid"]))
    axs[1, i].xaxis.set_ticks_position("bottom")
    axs[1, i].tick_params(axis="y", length=0, width=0, labelsize=6)
    axs[1, i].tick_params(axis="x", length=2, pad=0, labelsize=4)
    axs[1, i].xaxis.set_ticks([0, 20])
axs[1, 0].axes.get_yaxis().set_visible(False)
axs[1, 1].yaxis.set_ticks_position("right")

axs[1, 1].set_xlabel("[mmol/kg]", labelpad=1, fontsize=5)

plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.01, hspace=None)

plt.savefig("Cheese/Praxisproben/figures/chem_analysis.png", dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(4, 11))
spec = gridspec.GridSpec(
                        ncols=1, nrows=2, figure=fig,
                        height_ratios=[1, 8],
                        hspace=0.1, wspace=0)

ax1 = fig.add_subplot(spec[0, 0])
ax2 = fig.add_subplot(spec[1, 0])

chem = mpimg.imread("Cheese/Praxisproben/figures/chem_analysis.png")
heatmap = mpimg.imread('Cheese/Praxisproben/figures/qPCR_heatmap_copy.png')

ax1.imshow(chem, aspect="auto", interpolation='nearest')
ax2.imshow(heatmap, aspect="equal", interpolation='nearest')
pos1 = ax1.get_position()
pos1_new = [0.20, 0.72,  pos1.width/1.6, pos1.height] 
ax1.set_position(pos1_new)

ax1.set_frame_on(False)
ax1.patch.set_visible(False)
ax1.set_xticklabels([])
ax1.set_yticklabels([])
ax1.tick_params(axis="both",length=0, width=0., pad=0)

ax2.set_frame_on(False)
ax2.patch.set_visible(False)
ax2.set_xticklabels([])
ax2.set_yticklabels([])
ax2.tick_params(axis="both",length=0, width=0., pad=0)

plt.axis("off")

if not os.path.isdir("Figures"):
    os.mkdir("Figures")
plt.savefig("Figures/Fig5_cheese_defects.png", dpi=600, bbox_inches='tight')
plt.show()